In [242]:
import pandas as pd
import numpy as np

# Walk Through

Suppose you are testing three different headlines—A, B, and C—and you run them
each on 1,000 visitors, with the results shown in Table 3-4.

In [260]:
headline = pd.DataFrame({
    "headline a": [14, 986],
    "headline b": [8, 992],
    "headline c": [12, 988],
}, index=["click", "no click"])
headline

,headline a,headline b,headline c
click,14,8,12
no click,986,992,988


The headlines certainly appear to differ. Headline A returns nearly twice the click rate
of B. The actual numbers are small, though. 

A resampling procedure can test whether
the click rates differ to an extent greater than chance might cause. For this test, we
need to have the “expected” distribution of clicks, and in this case, that would be
under the null hypothesis assumption that all three headlines share the same click
rate, for an overall click rate of 34/3,000. Under this assumption, our contingency
table would look like Table 3-5.

In [261]:
headline_expected = pd.DataFrame({
    "headline a": np.mean(headline, axis=1), 
    "headline b": np.mean(headline, axis=1), 
    "headline c": np.mean(headline, axis=1), 
})
headline_expected

,headline a,headline b,headline c
click,11.333333,11.333333,11.333333
no click,988.666667,988.666667,988.666667


Now measure the pearson residual, which is defined as

$R = \frac{{\text{{Observed}} - \text{{Expected}}}}{{\sqrt{\text{{Expected}}}}}$

In [269]:
(headline.values - headline_expected.values) / np.sqrt(headline_expected)

,headline a,headline b,headline c
click,0.792118,-0.990148,0.198030
no click,-0.084809,0.106012,-0.021202


Then, the chi-square statistic is defined as the sum of the squared Pearson residuals.

In [279]:
np.sum(
    ((headline.values - headline_expected.values) / np.sqrt(headline_expected.values)) ** 2
)

1.6659394708658917

The chi-square statistic for this example is 1.666. Is that more than could reasonably occur in a chance
model? We can test with this resampling algorithm:
1. Constitute a box with 34 ones (clicks) and 2,966 zeros (no clicks).
2. Shuffle, take three separate samples of 1,000, and count the clicks in each.
3. Find the squared differences between the shuffled counts and the expected
counts and sum them.
4. Repeat steps 2 and 3, say, 1,000 times.
5. How often does the resampled sum of squared deviations exceed the observed?
That’s the p-value.

# With Resampling

In [300]:
observed = headline.values
expected = np.mean(observed, axis=1).reshape((2, 1))

def chi_square(observed, expected):
    pearson_res = (observed - expected) / np.sqrt(expected)
    chi_square_stat = np.sum(pearson_res ** 2)
    return chi_square_stat

observed_chi_square = chi_square(observed, expected)
print(f"Chi-square stats = {observed_chi_square}")

Chi-square stats = 1.6659394708658917


In [301]:
num_click = np.sum(headline, axis=1)["click"]
num_no_click = np.sum(headline, axis=1)["no click"]
box = np.array([1] * num_click + [0] * num_no_click)

def chi_square_permutation(box, num_iter=1000):

    chi_square_stats = []
    for _ in range(num_iter):
        # random shuffle the box
        np.random.shuffle(box)

        # sample for each headline, sum to get the number of click
        # assign 1000 samples to each headline by reshaping (3000, ) to (3, 1000)
        sample_clicks = box.reshape((3, -1)).sum(axis=1)
        # the remaining is no click
        sample_no_clicks = 1000 - sample_clicks

        # merge the samples, get the observed and expected
        all_samples = np.array([sample_clicks, sample_no_clicks])
        observed = all_samples
        expected = np.mean(observed, axis=1).reshape((2, 1))

        chi_square_stat = chi_square(observed, expected)
        chi_square_stats.append(chi_square_stat)
    return chi_square_stats

# this will return 10_000 chi square of resampled data
perm_chi_square = chi_square_permutation(box, 10_000)
# how often, in ratio, the resampled chi square exceed the onserved chi square
resampled_p_val = np.sum(perm_chi_square > observed_chi_square) / len(perm_chi_square)

print(f'Observed chi2: {observed_chi_square:.4f}')
print(f'Resampled p-value: {resampled_p_val:.4f}')

Observed chi2: 1.6659
Resampled p-value: 0.4609


p-value > 0.05. This means that we cannot reject H0.

# With Chi-Square Distribution

In [241]:
from scipy.stats import chi2_contingency

chisq, pvalue, df, expected = chi2_contingency(headline.values)
print(f'chi2: {chisq:.4f}')
print(f'p-value: {pvalue:.4f}')

chi2: 1.6659
p-value: 0.4348
